In [1]:
#ttps://eddwardo.github.io/posts/2019-06-05-finding-local-extreams-in-pandas-time-series/
import numpy as np
from scipy.signal import argrelextrema
from matplotlib import pyplot as plt
import pandas as pd
#%matplotlib inline
%matplotlib widget


In [2]:
min_period = 300
N = 14
df = pd.read_csv(f"data/USDT_BTC_{min_period}_1425168000_73.csv")
df.rename(columns={'date':'epoch','ts':'date'},inplace=True)
assert df.loc[0,'period'] == min_period, "Error, period different from min_period"
assert df['period'].count() == df.shape[0], "Error, period not always is min_period" 
df.drop(columns=['period','currency_pair','quoteVolume','weightedAverage'],inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by="date") #just in case
df.set_index('date',inplace=True)
df

,epoch,high,low,open,close,volume
date,,,,,,
2015-03-01 00:00:00,1425168000,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:05:00,1425168300,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:10:00,1425168600,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:15:00,1425168900,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:20:00,1425169200,240.000060,240.000060,240.000060,240.000060,0.000000
...,...,...,...,...,...,...
2021-03-31 23:35:00,1617233700,58802.587059,58761.857003,58801.000000,58762.000000,29415.195318
2021-03-31 23:40:00,1617234000,58798.268464,58744.683295,58762.000000,58795.050899,169054.589184
2021-03-31 23:45:00,1617234300,58866.551166,58795.050899,58795.050899,58810.585631,4315.762275


In [3]:
daily_df = df.resample('D').agg({'epoch':'first','high':'max','low':'min','open':'first','close':'last','volume':'sum'})

In [4]:
#daily_df = daily_df.loc["2020"]

In [5]:
#+- 4 days -> weekly extrema
i_supports = argrelextrema(daily_df.close.values, np.less_equal, order=4)[0] 
i_resistances = argrelextrema(daily_df.close.values, np.greater_equal, order=4)[0]

In [6]:
print(i_supports)
print(i_resistances)

[   0   17   25   44   56   66   67   80   95  115  139  153  160  170
  176  182  196  205  218  255  268  300  311  320  339  344  360  370
  384  395  404  423  428  436  448  479  494  509  520  533  545  560
  571  598  613  623  630  637  645  677  682  695  711  740  748  754
  774  818  837  848  853  868  878  885  904  918  928  936  948  962
  968  987  998 1027 1035 1047 1058 1072 1092 1112 1132 1167 1173 1184
 1191 1200 1209 1215 1230 1258 1268 1287 1296 1304 1312 1320 1328 1339
 1349 1366 1385 1397 1414 1422 1430 1439 1446 1456 1464 1473 1485 1502
 1516 1539 1556 1561 1583 1598 1609 1627 1635 1642 1654 1670 1680 1692
 1698 1705 1729 1739 1752 1761 1768 1791 1801 1817 1826 1838 1855 1872
 1877 1898 1911 1929 1945 1951 1964 1981 1990 2004 2013 2018 2033 2043
 2056 2098 2112 2122 2144 2153 2159 2191 2206 2216]
[   3    9   14   21   32   33   39   54   60   68   79   84  102  109
  133  150  155  181  190  201  211  247  258  274  289  299  307  312
  325  340  357  365  376

In [7]:
plt.style.use("seaborn")
fig1,ax1 = plt.subplots()

ax1.plot(daily_df.close,color="blue")
#daily_df.close.plot(color='blue')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#no funciona porque no son indices al ser df los indices date no ints
#df.loc[i_supports,'close'].plot(style='.', lw=10, color='red', marker="v")
#esto si funciona
#daily_df.iloc[i_resistances].close.plot(style='.', lw=10, color='red', marker="v")
ax1.plot(daily_df.iloc[i_resistances].close,linestyle='None', color='red', marker="v")

In [9]:
#daily_df.iloc[i_supports].close.plot(style='.', lw=10, color='green', marker="^")
ax1.plot(daily_df.iloc[i_supports].close,linestyle='None',  color='green', marker="^")

In [10]:
#green points labeled as buy, red points labeled as sell? Think about iloc and use with numpy and pandas datetime series
print(daily_df.iloc[i_resistances].close)

date
2015-03-04      280.000000
2015-03-10      291.000000
2015-03-15      282.108000
2015-03-22      267.516848
2015-04-02      255.000000
                  ...     
2021-01-08    40600.000000
2021-01-14    39158.756429
2021-02-21    57415.481351
2021-03-13    61179.998577
2021-03-30    58748.000000
Name: close, Length: 169, dtype: float64


In [15]:
print(daily_df.iloc[i_resistances].index)

DatetimeIndex(['2015-03-04', '2015-03-10', '2015-03-15', '2015-03-22',
               '2015-04-02', '2015-04-03', '2015-04-09', '2015-04-24',
               '2015-04-30', '2015-05-08',
               ...
               '2020-10-12', '2020-11-05', '2020-11-24', '2020-11-30',
               '2020-12-19', '2021-01-08', '2021-01-14', '2021-02-21',
               '2021-03-13', '2021-03-30'],
              dtype='datetime64[ns]', name='date', length=169, freq=None)


In [11]:
daily_df['label'] = 'stay'
#To transform list of ilocs to the index, use: df.iloc[ilocs_min].index
#daily_df.iloc[i_supports]['label'] = 'buy'
#daily_df.iloc[i_resistances]['label'] = 'sell'
daily_df.loc[daily_df.iloc[i_supports].index,'label'] = 'buy'
daily_df.loc[daily_df.iloc[i_resistances].index,'label'] = 'sell'



In [12]:
daily_df.to_csv("labeled_data.csv")